In [1]:
import scipy.io
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA,KernelPCA
from sklearn.grid_search import GridSearchCV
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#print(data.values())

/home/yzx/test/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/yzx/test/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### 读取数据集

In [72]:
L=[2,3,5,6,8,10,11,12,14]
num=[]
data=np.empty(shape=[0,200])
for i in L:
    tmp=scipy.io.loadmat('./train9/data'+str(i)+'_train.mat')['data'+str(i)+'_train']
    Len=tmp.shape[0]
    num.append(Len)
    data=np.append(data,tmp,axis=0)
#print(data)
num=np.array(num)

#print(sum(L))


 ### 设置数据集对应的标签

In [73]:
values=[]
for i in range(9):
    for j in range(num[i]):
        values.append(i)
values=np.array(values)

 ### 数据集划分成训练集和测试集,进行标准化预处理

In [74]:
x_train,x_test,y_train,y_test=train_test_split(data,values,test_size=0.2,random_state=5)
x_train=preprocessing.scale(x_train)
x_test=preprocessing.scale(x_test)

### gridsearch 调xgboost参数

In [80]:

'''
cv_params = {'max_depth':[x for x in range(1,20)],'min_child_weight':[x for x in range(1,10)], 'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}
other_params = {'max_depth':10,'min_child_weight':2, 'gamma':0.3,'learning_rate': 0.1, 'seed': 123, 'subsample': 0.8,'n_estimators':9, 'colsample_bytree': 0.8,'silent':1,'alpha':0.05}
model = xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(x_train, y_train)
evalute_result = optimized_GBM.grid_scores_
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
'''

"\ncv_params = {'max_depth':[x for x in range(1,20)],'min_child_weight':[x for x in range(1,10)], 'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}\nother_params = {'max_depth':10,'min_child_weight':2, 'gamma':0.3,'learning_rate': 0.1, 'seed': 123, 'subsample': 0.8,'n_estimators':9, 'colsample_bytree': 0.8,'silent':1,'alpha':0.05}\nmodel = xgb.XGBClassifier(**other_params)\noptimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=4)\noptimized_GBM.fit(x_train, y_train)\nevalute_result = optimized_GBM.grid_scores_\nprint('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))\nprint('最佳模型得分:{0}'.format(optimized_GBM.best_score_))\n"

### 训练xgboost模型

In [79]:
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 9,
    'gamma': 0.4,
    'max_depth': 11,
    'lambda': 2,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight':3,
    'silent': 1,
    'eta': 0.1,
    'seed': 100,
    'n_estimators':9,
    'alpha':0.05,
    'learning_rate': 0.1
}

plst=params.items()
Dtrain=xgb.DMatrix(x_train,y_train)
num_round=42
model=xgb.train(plst,Dtrain,num_round)

### 计算正确率

In [81]:
dtest = xgb.DMatrix(x_test)
ans = model.predict(dtest)
test_accuracy=accuracy_score(y_test,ans)
print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))

Test Accuracy: 89.46%


### 读取测试集并进行预测

In [82]:
Test=scipy.io.loadmat('./data_test_final.mat')['data_test_final']
Test=preprocessing.scale(Test)
DTest=xgb.DMatrix(Test)
ans=model.predict(DTest)
ANS=np.empty(shape=(len(ans),))
for i in range(len(ans)):
    ANS[i]=L[int(ans[i])]
#ANS 就是最后的结果集标签
print(ANS)

[6. 5. 3. ... 5. 3. 6.]


/home/yzx/test/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint16 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
